In [1]:
!pip -q install boto3

In [4]:
import boto3
import os
from collections import defaultdict

# -------------------------
# RunPod S3 Credentials
# -------------------------
ACCESS_KEY = "user_37sKcYrvnk9UXaIY3B3Zr90MH0g"
SECRET_KEY = "rps_YW72UMRXEMRVC8A407OCL08J8G34U1B3QTNO1ETX18pa1n"
BUCKET = "e9tcw5eupu"

ENDPOINT_URL = "https://s3api-eu-ro-1.runpod.io"
REGION = "eu-ro-1"  # required but not strictly enforced

# -------------------------
# S3 Client (IMPORTANT PART)
# -------------------------
s3 = boto3.client(
    "s3",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    endpoint_url=ENDPOINT_URL,
    region_name=REGION,
)

In [5]:
def human_size(bytes_size):
    if bytes_size >= 1024**3:
        return f"{bytes_size / 1024**3:.2f} GB"
    elif bytes_size >= 1024**2:
        return f"{bytes_size / 1024**2:.2f} MB"
    elif bytes_size >= 1024:
        return f"{bytes_size / 1024:.2f} KB"
    return f"{bytes_size} B"


def list_tree_with_sizes(bucket):
    paginator = s3.get_paginator("list_objects_v2")

    tree = defaultdict(list)
    folder_sizes = defaultdict(int)
    total_size = 0

    for page in paginator.paginate(Bucket=bucket):
        for obj in page.get("Contents", []):
            key = obj["Key"]
            size = obj["Size"]

            top_folder = key.split("/")[0]
            tree[top_folder].append((key, size))

            folder_sizes[top_folder] += size
            total_size += size

    # PRINT TREE
    print(f"\n📦 Bucket: {bucket}\n")

    for folder, files in sorted(tree.items()):
        print(f"📂 {folder}/  ({human_size(folder_sizes[folder])})")
        for key, size in files:
            print(f"   └── {key}  [{human_size(size)}]")

    print("\n" + "=" * 50)
    print(f"TOTAL SIZE: {human_size(total_size)}")
    print("=" * 50)


list_tree_with_sizes(BUCKET)


📦 Bucket: e9tcw5eupu

📂 data/  (355.24 MB)
   └── data/eff_training.csv  [355.24 MB]
📂 efficientnet/  (261.71 MB)
   └── efficientnet/eff_ann_version8.h5  [15.08 MB]
   └── efficientnet/efficientnetb7_feature_extractor.h5  [246.63 MB]
📂 sapiens/  (8.11 GB)
   └── sapiens/sapiens_2b_goliath_best_goliath_mIoU_8179_epoch_181_torchscript.pt2  [8.11 GB]
📂 scalers/  (119.96 KB)
   └── scalers/scaler_robust_features.pkl  [118.36 KB]
   └── scalers/scaler_standard_features.pkl  [593 B]
   └── scalers/scaler_targets.pkl  [1.02 KB]
📂 training-artifacts/  (166.35 KB)
   └── training-artifacts/IQR.pkl  [82.87 KB]
   └── training-artifacts/height_mean.pkl  [21 B]
   └── training-artifacts/meadian.pkl  [82.87 KB]
   └── training-artifacts/res_mean.pkl  [291 B]
   └── training-artifacts/res_std.pkl  [291 B]
   └── training-artifacts/std_height.pkl  [21 B]

TOTAL SIZE: 8.71 GB
